### Data Cleaning of 26,000 Records for Further Analysis

#### Packages Used

In [1]:
import re
import pandas as pd
import matplotlib.pyplot
import collections
from collections import Counter
import numpy as np
import seaborn as sns
import patsy
% pylab inline


Populating the interactive namespace from numpy and matplotlib


#### Import the file as DataFrame(df) 

In [2]:
df=pd.read_csv('data/Step3_input_raw_data.csv')

#### Start the Cleaning

In [3]:
#Remove Unwanted Columns:
df2=df.iloc[:,2:]
df2.head()

,body_style,drive_type,engine,ext_color,fuel,int_color,location,make,milage,model,mpg_city_highway,price,stock,transmission,vin,year,Region
0,SUV,FWD,6Cyl3.6L,Unspecified,Gasoline,Unspecified,"Indianapolis,IN",GMC,"90,440miles",Acadia,17/24,"$14,995",E79334,Automatic,1GKKRSED1BJ179334,":2011,",Central
1,Sedan,FWD,6Cyl3.5L,White,Flexible-Fuel,Black,"Fishers,IN",Chevrolet,"85,961miles",Impala,19/29,"$7,894",PV5110A,Automatic,2G1WF5EK1B1155818,":2011,",Central
2,Sedan,AWD,6Cyl2.5L,Gray,Gasoline,Black,"BeechGrove,IN",BMW,"152,000miles",3Series,17/24,"$4,200",m59833,Automatic,WBAEU334X4PM59833,":2004,",Central
3,Minivan,FWD,6Cyl3.5L,White,Gasoline,Tan,"Fishers,IN",Nissan,"40,347miles",Quest,19/25,"$21,200",18B378,Automatic,JN8AE2KP4E9106731,":2014,",Central
4,Van,FWD,4Cyl2.0L,White,Gasoline,Gray,"Fishers,IN",Nissan,"103,728miles",NV200,24/25,"$10,000",693736,Automatic,3N6CM0KN7DK693736,":2013,",Central


#### Extract Useful Columns

In [4]:
#Rearrange-cols
df3= df2[[ 'vin', 'price',
 'body_style',
 'drive_type',
 'engine',
 'ext_color',
 'fuel',
 'int_color',
 'location',
 'make',
 'milage',
 'model',
 'mpg_city_highway',
 'stock',
 'transmission',
 'year',
 'Region']]

In [5]:
df4 = pd.DataFrame(df3.location.str.split(',',).tolist(),
                                   columns = ['City','State'])

In [6]:

df4a = pd.DataFrame(df3.engine.str.split('Cyl',).tolist(),
                                   columns = ['Num_Cyl','Cyl_Volume'])

In [7]:
df5 = pd.concat([df4, df3,df4a], axis=1)

In [8]:
df5['price']=df5['price'].str.replace(',',"")
df5['price']=df5['price'].str.replace('$',"")
df5['Cyl_Volume']=df5['Cyl_Volume'].str.replace('L',"")

In [9]:
df6 = pd.DataFrame(df3.mpg_city_highway.str.split('/',).tolist(),
                                   columns = ['City_MPG','Highway_MPG'])


In [10]:
df7= pd.concat([df6, df5], axis=1)

In [11]:
df7['year']=df7['year'].str.replace(':',"")
df7['year']=df7['year'].str.replace(',',"")
df7['year']=df7['year'].str.replace(',',"")

In [12]:
df7['milage']=df7['milage'].str.replace('miles',"")
df7['milage']=df7['milage'].str.replace(',',"")

In [13]:
df7['engine']=df7['engine'].str.split('Cyl')

In [14]:
df8=df7.dropna()
df8.head()

,City_MPG,Highway_MPG,City,State,vin,price,body_style,drive_type,engine,ext_color,...,make,milage,model,mpg_city_highway,stock,transmission,year,Region,Num_Cyl,Cyl_Volume
0,17,24,Indianapolis,IN,1GKKRSED1BJ179334,14995,SUV,FWD,"[6, 3.6L]",Unspecified,...,GMC,90440,Acadia,17/24,E79334,Automatic,2011,Central,6,3.6
1,19,29,Fishers,IN,2G1WF5EK1B1155818,7894,Sedan,FWD,"[6, 3.5L]",White,...,Chevrolet,85961,Impala,19/29,PV5110A,Automatic,2011,Central,6,3.5
2,17,24,BeechGrove,IN,WBAEU334X4PM59833,4200,Sedan,AWD,"[6, 2.5L]",Gray,...,BMW,152000,3Series,17/24,m59833,Automatic,2004,Central,6,2.5
3,19,25,Fishers,IN,JN8AE2KP4E9106731,21200,Minivan,FWD,"[6, 3.5L]",White,...,Nissan,40347,Quest,19/25,18B378,Automatic,2014,Central,6,3.5
4,24,25,Fishers,IN,3N6CM0KN7DK693736,10000,Van,FWD,"[4, 2.0L]",White,...,Nissan,103728,NV200,24/25,693736,Automatic,2013,Central,4,2.0


In [15]:
#remove items without price
df9=df8[df8['price']!='CallforPrice']
df10=df9[df9['price']!='RequestQuote']

### Assign Variables to it's correct TYPE

In [16]:
df10[['price', 'City_MPG','milage','Highway_MPG','year']] = df10[['price', 'City_MPG','milage','Highway_MPG','year']].astype(int)

/Users/honeybadger4/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


### Visualize Data

In [17]:
df10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26061 entries, 0 to 26324
Data columns (total 23 columns):
City_MPG            26061 non-null int64
Highway_MPG         26061 non-null int64
City                26061 non-null object
State               26061 non-null object
vin                 26061 non-null object
price               26061 non-null int64
body_style          26061 non-null object
drive_type          26061 non-null object
engine              26061 non-null object
ext_color           26061 non-null object
fuel                26061 non-null object
int_color           26061 non-null object
location            26061 non-null object
make                26061 non-null object
milage              26061 non-null int64
model               26061 non-null object
mpg_city_highway    26061 non-null object
stock               26061 non-null object
transmission        26061 non-null object
year                26061 non-null int64
Region              26061 non-null object
Num_Cyl       

#### Validate Data

In [18]:

df10.groupby('year')['price'].sum().head(1)
# df10.to_csv('data/Step3_output_clean_df.csv')

#### Data Ready for Analysis:

In [19]:
#Cleaned Data TO ANALYZE!
xandy=df10[['price','milage','Highway_MPG','make', 'body_style','fuel', 'State','ext_color','int_color','Region','Num_Cyl','Cyl_Volume','year']]